# ccdproc-06:   Flatfielding

## Overview
1. Using master FLAT to correct from spatial variation of sensitivity all the images

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano and Nicolás Cardiel

This notebook has reproduced parts of the the astropy ccdproc docs: 
https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#subtract-bias-and-dark

Version 1.0 2021/01/16  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [ ]:
from pylab import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc

In [ ]:
# Some parameters for nicer graphs
matplotlib.rcParams['text.usetex']=False
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Verdana']
rcParams['font.size'] = 12.
rcParams['lines.linewidth'] = 4.
rcParams['axes.labelsize'] = 'large'
rcParams['grid.linewidth'] = 1.0
rcParams['grid.linestyle'] = ':'

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the working directory.  


At this step we have already created a combination of FLAT frames with the result of a several master FLATs (one for each filter used) that we called flat_N1_49.fits, flat_N1_76.fits and flat_N1_78.fits. This file should be in our working directory that also contents the trimmed images.  

In [ ]:
from ccdproc import ImageFileCollection
from ccdproc.utils.sample_directory import sample_directory_with_files

directory='/Users/jzamorano/Desktop/NOT_2008/N1/'   # change the path to your working directory

Let\'s create a list containing all the FITS files in that directory.  
We asume that the images were already trimmed and BIAS corrected and their names begin with 'zt_' (see previous notebooks 'ccdproc_02' and 'ccdproc_03')

In [ ]:
import os
from glob import glob
# os.path.join is a platform-independent way to join two directories
globpath = os.path.join(directory, 'zt_*.fits')
print(globpath)
# glob searches through directories similar to the Unix shell
filelist = sorted(glob(globpath))
print(filelist[100:105])    # printing only from 10 to 20

### Reading the master  FLATs files 

In [ ]:
master_flat_49 = CCDData.read(directory+'flat_N1_49.fits')
master_flat_76 = CCDData.read(directory+'flat_N1_76.fits')
master_flat_78 = CCDData.read(directory+'flat_N1_78.fits')

## Correcting all files from FLAT

### Using ccdproc_correct_flat

In [ ]:
keys = ['OBJECT' , 'EXPTIME' , 'ALFLTID' , 'FAFLTID' , 'FBFLTID']
ic1 = ImageFileCollection(directory, keywords=keys) # only keep track of keys
ic1.summary.colnames

In [ ]:
matches = (ic1.summary['FBFLTID'] == 78)
sci_list_78 = ic1.summary['file'][matches]
print(sci_list_78)

In [ ]:
sci_images_78 = []
for file in sci_list_78:
    sci_images_78.append(CCDData.read(directory+file)) 

In [ ]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for i in range(len(sci_list_78)):
    print(i,sci_images_78[i].header['FILENAME'], 
          sci_images_78[i].header['OBJECT'], 
          sci_images_78[i].header['EXPTIME'], 
          int(np.mean(sci_images_78[i])), 
          int(np.std(sci_images_78[i])), 
          np.min(sci_images_78[i]), 
          np.max(sci_images_78[i]))

In [ ]:
print(sci_list_78[19:26])   # interesting sci images in filter #78

In [ ]:
sci_list_78 = sci_list_78[19:26]
print(sci_list_78)

In [ ]:
for file in sci_list_78:
    image = CCDData.read(directory+file)
    f_image = ccdproc.flat_correct(image,master_flat_78)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_78.fits master FLAT' 
    print('writting '+name_of_file+ ' in '+directory)
    f_image.write(directory+name_of_file,overwrite='yes')

In [ ]:
matches = (ic1.summary['ALFLTID'] == 76)
sci_list_76 = ic1.summary['file'][matches]
print(sci_list_76)

In [ ]:
sci_images_76 = []
for file in sci_list_76:
    sci_images_76.append(CCDData.read(directory+file)) 

In [ ]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for i in range(len(sci_list_76)):
    print(i,sci_images_76[i].header['FILENAME'], 
          sci_images_76[i].header['OBJECT'], 
          sci_images_76[i].header['EXPTIME'], 
          int(np.mean(sci_images_76[i])), 
          int(np.std(sci_images_76[i])), 
          np.min(sci_images_76[i]), 
          np.max(sci_images_76[i]))

In [ ]:
print(sci_list_76[17:41])   # interesting sci images in filter #76

In [ ]:
sci_list_76 = sci_list_76[17:41]
print(sci_list_76)

In [ ]:
for file in sci_list_76:
    image = CCDData.read(directory+file)
    f_image = ccdproc.flat_correct(image,master_flat_76)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_76.fits master FLAT' 
    print('writting '+name_of_file+ ' in '+directory)
    f_image.write(directory+name_of_file,overwrite='yes')

In [ ]:
matches = (ic1.summary['FBFLTID'] == 49)
sci_list_49 = ic1.summary['file'][matches]
print(sci_list_49)

In [ ]:
sci_images_49 = []
for file in sci_list_49:
    sci_images_49.append(CCDData.read(directory+file)) 

In [ ]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for i in range(len(sci_list_49)):
    print(i,sci_images_49[i].header['FILENAME'], 
          sci_images_49[i].header['OBJECT'], 
          sci_images_49[i].header['EXPTIME'], 
          int(np.mean(sci_images_49[i])), 
          int(np.std(sci_images_49[i])), 
          np.min(sci_images_49[i]), 
          np.max(sci_images_49[i]))

In [ ]:
print(sci_list_49[19:27])   # interesting sci images in filter #76

In [ ]:
sci_list_49 = sci_list_49[19:27]
print(sci_list_49)

In [ ]:
for file in sci_list_49:
    image = CCDData.read(directory+file)
    f_image = ccdproc.flat_correct(image,master_flat_49)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_49.fits master FLAT' 
    print('writting '+name_of_file+ ' in '+directory)
    f_image.write(directory+name_of_file,overwrite='yes')